In [10]:
import cv2
import mediapipe as mp
from math import hypot
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import numpy as np

In [11]:
# Access Camera
cap = cv2.VideoCapture(0)

# Detecting, Initializing and configuring the hands

mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_draw = mp.solutions.drawing_utils

In [12]:
# Accessing Speaker using Pycaw

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface,POINTER(IAudioEndpointVolume))

In [13]:
# Finding volume range

vol_min, vol_max = volume.GetVolumeRange()[:2]

In [ ]:
# Capture an Image from the Camera

while True:
    success, img = cap.read()
    img_RGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(img_RGB)

    # Checking for multiple hands
    lm_list = []
    if results.multi_hand_landmarks:
        for hand_land_mark in results.multi_hand_landmarks:
            for id,lm in enumerate(hand_land_mark.landmark):
                h,w,_ = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                lm_list.append([id,cx,cy])
            mp_draw.draw_landmarks(img, hand_land_mark, mp_hands.HAND_CONNECTIONS)
    
    if lm_list != []:
        x1, y1 = lm_list[4][1], lm_list[4][2]
        x2, y2 = lm_list[8][1], lm_list[8][2]

        # Drawing a circle at the tip of the index finger and thumb
        cv2.circle(img, (x1,y1), 15, (255,0,0),cv2.FILLED)
        cv2.circle(img, (x2,y2), 15, (255,0,0),cv2.FILLED)
        
        # Drawing a line between the fingertips
        cv2.line(img, (x1,y1), (x2,y2), (0,255,0), 3)
        # Calculating the distance between the index finger and thumb
        length = hypot(x2-x1, y2-y1)

        # Converting finger range to volume range

        vol = np.interp(length, [15,220], [vol_min, vol_max])
        print(vol,length)

        # Setting the volume
        volume.SetMasterVolumeLevel(vol, None)

    cv2.imshow('img', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
